## Test Code for the cGAN

In [8]:
import torch
import pathlib
def celeb_label_transform(desired_attr):
    """
    Pass in a the names of all the attributes that you want
    """

    file = open('C:/Users/Paddy/CRT/Github/input/CelebA/list_attr_celeba.csv').read().split()
    attr_names = file[0].split(',')
    file = file[1:]
    
    def transform(idx):
        attr = torch.tensor([int(entry) for entry in file[idx].split(',')[1:]])
        mask = [attr_names[1:][i] in desired_attr for i in range(len(attr))]
        masked = attr[mask]
        return torch.relu(masked).float()
    return transform

file = open('C:/Users/Paddy/CRT/Github/input/CelebA/list_attr_celeba.csv').read().split()
attr_names = file[0].split(',')

bald_transform = celeb_label_transform("Bald")
bald_transform(50)


tensor([1.])

In [9]:
import pandas as pd

targ_dir = 'C:/Users/Paddy/CRT/Github/input/SEN12MS/'
filenames = list(pathlib.Path(targ_dir).glob("*/*/*.tif"))

# dictionary of lists
dict = {'name': filenames}
     
df = pd.DataFrame(filenames)
     
# saving the dataframe
df.to_csv('sen12_filenames.csv')

In [40]:
import torch

def sen12_label_transform(desired_season):
    """
    Pass in a the names of all the attributes that you want
    """

    file = open('C:/Users/Paddy/CRT/Github/input/SEN12MS/seasons_labeled_spring.csv').read().split()
    season_names = file[0].split(',')
    file = file[1:]
    
    def transform(idx):
        season = torch.tensor([int(entry) for entry in file[idx].split(',')[1:]])
        mask = [season_names[1:][i] in desired_season for i in range(len(season))]
        masked = season[mask]
        return torch.relu(masked).float()
    return transform

tensor([1.])

In [36]:
import os
import pandas as pd
import pathlib
from tqdm import tqdm

path = "C:/Users/Paddy/CRT/Github/input/SEN12MS/"
filenames = list(pathlib.Path(path).glob("*/*/*.tif"))

path = "C:/Users/Paddy/CRT/Github/input/SEN12MS/ROIs1158_spring"
# s1 = fall, s6 = spring

foldernames = list(pathlib.Path(path).glob("*/"))

seasons_original = pd.read_csv('C:/Users/Paddy/CRT/Github/input/SEN12MS/seasons_spring.csv')

seasons = []
filenames = []

for folder in foldernames:
    for file in list(pathlib.Path(folder).glob("*.tif")):
        season_ref = "_".join(os.path.basename(file).split("_")[0:4])
        season = seasons_original.loc[seasons_original["scene"] == season_ref]["true_season"]
        filenames.append(os.path.basename(file))
        seasons.append(season.item())

spring = [1 if x == 'spring' else -1 for x in seasons]
summer = [1 if x == 'summer' else -1 for x in seasons]
fall = [1 if x == 'fall' else -1 for x in seasons]
winter = [1 if x == 'winter' else -1 for x in seasons]

df = pd.DataFrame(zip(filenames, spring, summer, fall, winter), columns=['scene','spring', 'summer', 'fall', 'winter'])

print("Saving data...")
df.to_csv("seasons_labeled_spring.csv", index=False)
print("Saved.")
print(df.head())

Saving data...
Saved.
                           scene  spring  summer  fall  winter
0  ROIs1158_spring_s2_1_p100.tif      -1      -1     1      -1
1  ROIs1158_spring_s2_1_p101.tif      -1      -1     1      -1
2  ROIs1158_spring_s2_1_p102.tif      -1      -1     1      -1
3  ROIs1158_spring_s2_1_p103.tif      -1      -1     1      -1
4  ROIs1158_spring_s2_1_p104.tif      -1      -1     1      -1


In [2]:
import os
import pandas as pd
import pathlib
from tqdm import tqdm

path = "C:/Users/Paddy/CRT/Github/input/SEN12MS/"
filenames = list(pathlib.Path(path).glob("*/*/*.tif"))

print(filenames[0])

C:\Users\Paddy\CRT\Github\input\SEN12MS\ROIs1158_spring\s2_1\ROIs1158_spring_s2_1_p100.tif


In [9]:
import torch
import torchvision.transforms as transforms
import torch.optim as optim
from model import *
from transforms import *
from training_loop import *
from celeba_data import *
from sen12_data import *
from torch.utils.data import DataLoader
import os
import argparse
from torchinfo import summary
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

topn = 1
checkpoint_dir = os.path.join(os.path.dirname(os.getcwd()), "checkpoints")
name = 'sen12_cgan_test'
batch_size = 32
gen_steps = 1
disc_steps = 1
epochs = 1
img_size = 256
lr = 0.0002
beta = 0.5
desired_season = ['fall']
desired_attr = ['Young']
label_size = len(desired_season)
data_source = "C:/Users/Paddy/CRT/Github/input/SEN12MS"
#data_source = "C:/Users/Paddy/CRT/Github/input/sen12_overfit/ROIs1158_spring_s2_1_p30.tif"
source_labels = "C:/Users/Paddy/CRT/Github/input/SEN12MS/seasons_labeled_spring.csv"
#source_labels = "C:/Users/Paddy/CRT/Github/input/sen12_overfit/seasons_labeled_overfit.csv"
bands = "rgb" # or "rgb"
img_channels = 3

transform_sen = transforms.Compose(
[
    transforms.ToTensor(),
    transforms.Resize((img_size,img_size),antialias=False),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Normalize([0.5 for _ in range(img_channels)],[0.5 for _ in range(img_channels)])
])
anntransform_celeb = celeb_label_transform(desired_attr)
anntransform_sen12 = sen12_label_transform(source_labels, desired_season)
anntransform_sen12_overfit = sen12_overfit_label_transform(source_labels, desired_season)
anntransform_celeb_overfit = celeb_label_transform_overfit(desired_attr)

imgtransform = BasicImageCropTransform(size = (img_size, img_size), scale = (1, 2))

dataset_sen12 = SEN12MS(data_source, transform_sen, anntransform_sen12, "rgb")
dataset_sen12_overfit = SEN12MS_overfit(data_source, transform_sen, anntransform_sen12_overfit, "rgb", 200000)
dataset_celeba = CelebDS(imgtransform, anntransform_celeb)
dataset_celeba_overfit = CelebDS_overfit(imgtransform, anntransform_celeb_overfit, 50000)

first_celeb = dataset_celeba[0][0]
first_sen12 = dataset_sen12[0][0]

print("CelebA Input dimenstion:", first_celeb)
print("Max = ", torch.max(first_celeb).dtype)
print("Min = ", torch.min(first_celeb).dtype)

CelebA Input dimenstion: tensor([[[ 0.8648,  1.0000,  1.0000,  ...,  0.8479,  0.9312,  0.8261],
         [ 1.0000,  1.0000,  0.8706,  ...,  1.0000,  1.0000,  0.9656],
         [ 0.9470,  0.9622,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
         ...,
         [ 0.1515,  0.3773,  0.5001,  ...,  0.4716,  0.3428,  0.1306],
         [ 0.2813,  0.3296,  0.4257,  ...,  0.4941,  0.4790,  0.3756],
         [ 0.3725,  0.4545,  0.3997,  ...,  0.5082,  0.5082,  0.6464]],

        [[ 0.6321,  0.8289,  0.8662,  ...,  1.0000,  0.9756,  0.9149],
         [ 0.7770,  0.9125,  0.7045,  ...,  0.9733,  0.9010,  1.0000],
         [ 1.0000,  0.8156,  0.7264,  ...,  0.8327,  0.8453,  0.8458],
         ...,
         [-0.2444, -0.1075, -0.1685,  ..., -0.1184, -0.0758, -0.0397],
         [-0.1962, -0.3074, -0.1081,  ..., -0.0409,  0.0083, -0.0622],
         [-0.0560, -0.1942, -0.0068,  ...,  0.0481,  0.0845,  0.1234]],

        [[ 0.8608,  0.5511,  0.4250,  ...,  0.8921,  0.5854,  0.8252],
         [ 0.7108,  

In [16]:
print("Sen12 Input dimenstion:", first_sen12)
print("Max = ", torch.max(first_sen12))
print("Min = ", torch.min(first_sen12))
#print("Sen12MS Input dimenstion:", dataset_sen12[0][0])

Sen12 Input dimenstion: torch.Size([256, 256])
Max =  tensor(1.)
Min =  tensor(-1.)
